# Training FCN Model
This script performs the training of a network after the dataset has been constructed. Four modules are used in this training:

1. `input_data.py`: This module loads in the training and testing data that was created, and prepares it so that it can be fed into the neural network for training.
2. `models.py`: this module contains different types of neural network architectures. One can choose a model from this script and train it.
3. `training_utilities.py`: this module contains functions to construct models, set up diagnostic files, and perform the training on the model.
4. `accuracy.py`: The accuracy that the FCN outputs looks at the predicted label and evaluated label, and does a pixelwise comparison (#pixels set to the correct value)/(#pixels in the image). This isn't the best way to determine the success of our model. Rather, we want to look at a metric that shows that the model is detecting defects (where each defect consists of multiple pixels). Hence we use measures such as the recall, which is the number of true positive defect detections TN over the true positives plus the false negatives (FN), R = TN/(TN+FN). This module helps in calculating such metrics

First, we import these modules

In [ ]:
from input_data import *
from models import *
from training_utilities import *

## Parameters ##
Next, we need to provide parameter information based on the data and model we're using.

**parent_dir**: This is the path to the "stem-learning" code

**data_dir**: This is the path to the data that we created in the preprocessing section

**sess_name**: we will create a folder in the "results" directory called session_name, where all the output will be stored

**N**: This is the pixel width/height of the input images (note we're assuming a square image)

**k_fac**: this is a factor that describes how many channels we want per layer in our FCNs. Whatever the default value is per layer, it is multiplied by k_fac.

**nb_classes**: this is the number of labels that we are learning at once. For example, if our data is just the "2Te" labels, then nb_classes = 2 (2Te and no defect). 

**dropout**: This is the dropout rate at the end of our FCN

**num_steps**: The total number of steps to train on (-1 for infinite loop)

In [ ]:
parent_dir = 'D:/stem-learning/'
identifier = "20220525_unet_dist_gen_fft_10_pristine_gaussian_0.1"
sess_name  = '1vacancy'
data_dir = parent_dir + "data/WSe/data_for_gan/generated/GANNED_DATA_FOLDER/data_folder_{}/parsed_label_{}/".format(identifier, sess_name)
N          = 256
k_fac      = 16
nb_classes = 2
num_steps  = 200

The variables below are then created to locate the directories that we'll be storing our data

In [ ]:
from os import makedirs
sess_dir = "{}results/results_{}/{}/".format(parent_dir, identifier, sess_name)
makedirs(sess_dir, exist_ok=True)

model_weights_fn = sess_dir + "weights.h5"
model_fn         = sess_dir + "model.json"
diagnostics_fn   = sess_dir + "diagnostics.dat"

Now we create the model and set up a diagnostics file

In [ ]:
model = construct_model(N, k_fac, nb_classes, sess_dir, model_fn, model_weights_fn)
step = setup_diagnostics(diagnostics_fn)

Finally, we train

In [ ]:
train(step, data_dir, N, nb_classes, model, diagnostics_fn, model_weights_fn, num_steps=num_steps)